In [1]:
import tensorflow as tf

In [2]:
a = tf.Variable(0)
b = tf.Variable(1)
pair = tf.Variable(b)
people_per_batch = tf.constant(10)
a_idx = tf.Variable(0, dtype=tf.int32)
p_idx = tf.Variable(0, dtype=tf.int32)
_nrof_images_per_class = [5 for i in xrange(10)]
print(_nrof_images_per_class)
nrof_images_per_class = tf.constant(_nrof_images_per_class)
emb_start_idx = tf.Variable(0)
triplets = tf.constant([-1,-1,-1,-1,-1,-1], shape=[6,1], name='triplets')
print(triplets)

def update_i():
    return tf.cond(a.assign_add(1) < people_per_batch, lambda:b.assign(1), lambda:b.assign(1))

def update_b_and_pair():
    return tf.cond((b+1) < nrof_images_per_class[a], lambda: b.assign_add(1), update_i)

def condition(a, b, pair, emb_start_idx, a_idx, p_idx, triplets):
    r = tf.less(a, people_per_batch-1)
    return r 

def body(a, b, pair, emb_start_idx, a_idx, p_idx, triplets):
    
    a_idx = tf.add(emb_start_idx, tf.subtract(b,1))#
    p_idx = tf.add(emb_start_idx, pair)
    triplet = tf.stack([a, b, pair, emb_start_idx, a_idx, p_idx], name='triplet')
    triplet = tf.reshape(triplet, shape=(6,1))
    triplets = tf.concat([triplets, triplet], axis=1)
    
    #triplet = tf.stack([a_idx, p_idx], name='triplet')
    #triplet = tf.reshape(triplet, shape=(2,1))
    #triplets = tf.concat([triplets, triplet], axis=1)
    #emb_start_idx = tf.cond( (b+1) < nrof_images_per_class[a], 
                            #lambda: emb_start_idx, lambda: tf.add(emb_start_idx ,nrof_images_per_class[a]))
    pair = tf.cond((pair+1) < nrof_images_per_class[a], lambda: pair+1, update_b_and_pair)
    
    return a, b, pair, emb_start_idx, a_idx, p_idx, triplets

[a, b, pair, emb_start_idx, a_idx, p_idx, triplets] = tf.while_loop(condition, body, 
                                                      loop_vars=[a, b, pair, emb_start_idx, a_idx, p_idx, triplets],
                                                     shape_invariants=[a.shape, b.shape, 
                                                                       pair.shape, emb_start_idx.shape,
                                                                       a_idx.shape, p_idx.shape,
                                                                      tf.TensorShape([6,None])])

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Tensor("triplets:0", shape=(6, 1), dtype=int32)


In [3]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

In [4]:
triplets = sess.run(triplets)
print(triplets.T)

[[-1 -1 -1 -1 -1 -1]
 [ 0  1  1  0  0  1]
 [ 0  1  2  0  0  2]
 [ 0  1  3  0  0  3]
 [ 0  1  4  0  0  4]
 [ 0  2  2  0  0  2]
 [ 0  2  3  0  1  3]
 [ 0  2  4  0  1  4]
 [ 0  3  3  0  1  3]
 [ 0  3  4  0  1  4]
 [ 0  4  4  0  2  4]
 [ 1  1  1  0  2  1]
 [ 1  1  2  0  2  2]
 [ 1  1  3  0  3  3]
 [ 1  1  4  0  3  4]
 [ 1  2  2  0  0  2]
 [ 1  2  3  0  0  3]
 [ 1  2  4  0  0  4]
 [ 1  3  3  0  0  3]
 [ 1  3  4  0  0  4]
 [ 1  4  4  0  1  4]
 [ 2  1  1  0  1  1]
 [ 2  1  2  0  1  2]
 [ 2  1  3  0  2  3]
 [ 2  1  4  0  2  4]
 [ 2  2  2  0  2  2]
 [ 2  2  3  0  3  3]
 [ 2  2  4  0  3  4]
 [ 2  3  3  0  3  3]
 [ 2  3  4  0  0  4]
 [ 2  4  4  0  0  4]
 [ 3  1  1  0  0  1]
 [ 3  1  2  0  0  2]
 [ 3  1  3  0  1  3]
 [ 3  1  4  0  1  4]
 [ 3  2  2  0  1  2]
 [ 3  2  3  0  2  3]
 [ 3  2  4  0  2  4]
 [ 3  3  3  0  3  3]
 [ 3  3  4  0  3  4]
 [ 3  4  4  0  0  4]
 [ 4  1  1  0  0  1]
 [ 4  1  2  0  0  2]
 [ 4  1  3  0  1  3]
 [ 4  1  4  0  1  4]
 [ 4  2  2  0  1  2]
 [ 4  2  3  0  2  3]
 [ 4  2  4  0